In [1]:
import os 
import sys

sys.path.append('..')



c:\Users\Conner\anaconda3\envs\athought\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Registering agent: PineconeAgent


In [29]:
from src.agents.base_agent import ChainableAgent, register_chain
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


from src.agents.agent_registry import register_agent


import json

def create_debug_wrapper(name=""):
    def debug_wrapper(x):
        print(f"\n--- Debug: {name} Input ---")
        print(json.dumps(x, indent=2, default=str))
        return x  # Just pass through the input
    return RunnableLambda(debug_wrapper)

class PineconeAgent():
    def __init__(self, title, **kwargs):
        self.embeddings = OpenAIEmbeddings()
        self.vector_store = PineconeVectorStore(index_name='langchain-test-index', embedding=self.embeddings)
        self.retriever = self.vector_store.as_retriever()

    def rag_chain(self):
        rag_prompt_template = """Use the following pieces of context to answer the question at the end. 
        If you don't know the answer, just say that you don't know, don't try to make up an answer.
        Use three sentences maximum and keep the answer as concise as possible.
        {context}
        Query: {query}
        Helpful Answer:"""

        rag_prompt = PromptTemplate.from_template(rag_prompt_template)

        def get_context(inputs):
            query = inputs["query"]
            docs = self.retriever.invoke(query)
            return "\n\n".join(doc.page_content for doc in docs)

        rag_chain = (
            # RunnablePassthrough.assign(context=RunnableLambda(get_context), query=itemgetter('query'))
            RunnablePassthrough.assign(context=RunnableLambda(get_context))
            | create_debug_wrapper("Context and Query")
            | rag_prompt 
            | create_debug_wrapper("RAG Prompt")
            | ChatOpenAI(model='gpt-4o-2024-05-13')
            | create_debug_wrapper("ChatBot Response")
            | StrOutputParser()
        )

        return rag_chain



In [30]:
agent = PineconeAgent(title='RAG Agent')
rc = agent.rag_chain()


In [31]:
answer = rc.invoke({'query': 'Why is langchain so popular?', 'role': 'You are a helpful assistant'})



--- Debug: Context and Query Input ---
{
  "query": "Why is langchain so popular?",
  "role": "You are a helpful assistant",
  "context": "LangChain, which really helped it catch on. The platform formed a strong community on Discord,\nwhere people continue to have lively discussions. In April 2023, LangChain attracted some big\nventure capital investments for around $20 million. LangChain provides pre-built components\ntailored for typical use cases, such as summarization, question & answer, and multi-agent bots.\n\u00a0\u00a0\u00a0\u00a0\u00a01.1.1 LangChain architecture\n\n1.1 Introducing LangChain\n\u00a0\u00a0\u00a0\u00a0Since late 2022, several open-source LLM application frameworks have emerged. These frameworks\no\ufb00er ready-made components for common functions needed in developing LLM-powered\napplications, saving developers time and e\ufb00ort. They stay current with updates in data sources,\nvector stores, and LLM advancements, thanks to collaboration from hundreds of dev

In [19]:
answer

'The moon does not have a capital as it is not a governed entity.'